In [1]:
from obspy import read
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os

In [10]:
# Load the pick data
pick_data = pd.read_csv(
    "dataset_earthquakes/metadata.csv", 
    usecols=['trace_name_original_Z', 'trace_p_pick_time', 'trace_s_pick_time', 'source_sensor_distance']
)
# pick_data

In [11]:
# Function to sanitize file names
def sanitize_filename(name):
    return name.replace(".", "").replace(":", "").replace("-", "").replace("_", "")

# Directory containing miniSEED files
mseed_dir = "miniSEED_files"
output_dir = "output_plots_srcdist"
os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist


In [12]:
# Loop through each miniSEED file in the directory
for file_path in Path(mseed_dir).glob("*.MSEED"):
    mseed_name = sanitize_filename(file_path.name.strip())  # Sanitize the miniSEED file name

    # Check if the miniSEED file matches an entry in the CSV
    matched_row = None
    for _, row in pick_data.iterrows():
        if sanitize_filename(row['trace_name_original_Z']) == mseed_name:
            matched_row = row
            break

    if matched_row is None:
        print(f"No match found for file: {file_path.name}")
        continue

    print(f"Match found: {file_path.name}")

    # Extract P and S pick times
    try:
        p_pick = UTCDateTime(matched_row['trace_p_pick_time'])
        s_pick = UTCDateTime(matched_row['trace_s_pick_time'])
        source_sensor_distance = matched_row['source_sensor_distance']  # Extract the distance
    except Exception as e:
        print(f"Error parsing data for {file_path.name}: {e}")
        continue

    # Read the miniSEED file
    stream = read(file_path)

    # Plot and save each trace in the stream
    for trace in stream:
        fig, ax = plt.subplots(figsize=(10, 6))

        # Plot the waveform
        times = trace.times("matplotlib")
        ax.plot(times, trace.data, label=f"Waveform: {trace.id}", color="#008080")

        # Add vertical lines for P and S picks
        ax.axvline(p_pick.matplotlib_date, color="red", linestyle="--", label="P-wave Pick")
        ax.axvline(s_pick.matplotlib_date, color="black", linestyle="--", label="S-wave Pick")

        # Customize the plot
        ax.set_title(f"Waveform and Picks: {trace.id}\nDistance: {source_sensor_distance} km")
        ax.set_xlabel("Time (UTC)")
        ax.set_ylabel("Amplitude")
        ax.legend()
        ax.grid()

        # Format the x-axis for date/time
        fig.autofmt_xdate()

        # Save the plot
        output_file = Path(output_dir) / f"{file_path.stem}_{trace.id}.png"
        plt.savefig(output_file)
        plt.close()
        print(f"Plot saved to: {output_file}")

No match found for file: 34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DN1.MSEED
No match found for file: 34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DN2.MSEED
Match found: 34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DNZ.MSEED
Plot saved to: output_plots_srcdist\34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DNZ_WS.POZA.S5.DNZ.png
No match found for file: 34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DN1.MSEED
No match found for file: 34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DN2.MSEED
Match found: 34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DNZ.MSEED
Plot saved to: output_plots_srcdist\34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DNZ_WS.POZA.S3.DNZ.png
No match found for file: 34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DN1.MSEED
No match found for file: 34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DN2.MSEED
Match found: 34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DNZ.MSEED
Plot saved to: output_plots_srcdist\34161341_2023-02-21T00.07.00.490032Z_WS

In [5]:
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import trigger_onset
from obspy.core.trace import Trace

# Define STA and LTA windows (in seconds)
sta_window = 1  # Short-Term Average window (e.g., 1 second)
lta_window = 10  # Long-Term Average window (e.g., 10 seconds)

# Sampling rate (assumes all traces have the same sampling rate)
sampling_rate = trace.stats.sampling_rate  # Get sampling rate from the trace

# Calculate STA/LTA for each trace in the stream
for trace in stream:
    print(f"Processing trace: {trace.id}")

    # Apply STA/LTA algorithm
    cft = classic_sta_lta(trace.data, int(sta_window * sampling_rate), int(lta_window * sampling_rate))

    # Plot the STA/LTA ratio
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot the waveform
    times = trace.times("matplotlib")
    ax1.plot(times, trace.data, label=f"Waveform: {trace.id}", color="blue", alpha=0.6)
    ax1.set_xlabel("Time (UTC)")
    ax1.set_ylabel("Amplitude")
    ax1.set_title(f"Waveform and STA/LTA: {trace.id}")
    ax1.legend(loc="upper left")
    ax1.grid()

    # Plot the STA/LTA ratio
    ax2 = ax1.twinx()
    ax2.plot(times[:len(cft)], cft, label="STA/LTA Ratio", color="orange")
    ax2.set_ylabel("STA/LTA Ratio")
    ax2.legend(loc="upper right")

    # Show P and S picks on the waveform
    ax1.axvline(p_pick.matplotlib_date, color="red", linestyle="--", label="P-wave Pick")
    ax1.axvline(s_pick.matplotlib_date, color="black", linestyle="--", label="S-wave Pick")

    # Save the plot
    output_file = Path(output_dir) / f"{file_path.stem}_{trace.id}_sta_lta.png"
    plt.savefig(output_file)
    plt.close()
    print(f"STA/LTA plot saved to: {output_file}")


Processing trace: WS.POZA.S4.DN1
STA/LTA plot saved to: output_plots_srcdist\35419441_2023-06-30T04.10.40.109398Z_WS.POZA.S4.DNZ_WS.POZA.S4.DN1_sta_lta.png
